## Recommended System: Movies

In [143]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt

In [159]:
df = pd.read_csv('d:\\Desktop/mrating.csv')
df

,NAME,Dr. strange,Minions,Thor,Top Gun,Bullet Train,Moon Man
0,Liu,3.0,2.0,3.0,2.0,5.0,5.0
1,Kris,3.0,1.0,2.0,3.0,4.0,NaN
2,Mike,2.0,4.0,3.0,NaN,3.0,NaN
3,Mario,NaN,1.0,NaN,NaN,NaN,4.0
4,Leo,3.0,NaN,2.0,NaN,3.0,NaN


In [194]:
df.mean()

Dr. strange     2.75
Minions         2.00
Thor            2.50
Top Gun         2.50
Bullet Train    3.75
Moon Man        4.50
dtype: float64

In [203]:
# Conver to long format
df_rating = df.melt(id_vars = ['NAME'], value_vars = ['Dr. strange','Minions','Thor','Bullet Train','Top Gun','Moon Man'])
df_rating

,NAME,variable,value
0,Liu,Dr. strange,3.0
1,Kris,Dr. strange,3.0
2,Mike,Dr. strange,2.0
3,Mario,Dr. strange,NaN
4,Leo,Dr. strange,3.0
5,Liu,Minions,2.0
6,Kris,Minions,1.0
7,Mike,Minions,4.0
8,Mario,Minions,1.0
9,Leo,Minions,NaN


In [230]:
userRatings = df_rating.pivot_table(index=['NAME'],columns=['variable'],values='value')
userRatings

variable,Bullet Train,Dr. strange,Minions,Moon Man,Thor,Top Gun
NAME,,,,,,
Kris,4.0,3.0,1.0,NaN,2.0,3.0
Leo,3.0,3.0,NaN,NaN,2.0,NaN
Liu,5.0,3.0,2.0,5.0,3.0,2.0
Mario,NaN,NaN,1.0,4.0,NaN,NaN
Mike,3.0,2.0,4.0,NaN,3.0,NaN


In [180]:
corrMatrix = df.corr(method='pearson').fillna(0)
corrMatrix

,Dr. strange,Minions,Thor,Top Gun,Bullet Train,Moon Man
Dr. strange,1.000000,-0.944911,-0.577350,0.0,0.522233,0.0
Minions,-0.944911,1.000000,0.755929,-1.0,-0.654654,1.0
Thor,-0.577350,0.755929,1.000000,-1.0,0.301511,0.0
Top Gun,0.000000,-1.000000,-1.000000,1.0,-1.000000,0.0
Bullet Train,0.522233,-0.654654,0.301511,-1.0,1.000000,0.0
Moon Man,0.000000,1.000000,0.000000,0.0,0.000000,1.0


In [234]:
def recommend_movie(NAME, hm):
    myRatings = userRatings.loc[NAME].dropna()
    similar_candidates = pd.DataFrame()
    for i in list(corrMatrix.index):
        # retrieve similar movies for movie i
        similar_movies = corrMatrix[i]
        # substract to similar score between movie i and rated movies
        similar_movies = similar_movies[similar_movies.index.isin(myRatings.index)]
        # calculate predict rating
        predict_ratings = sum(myRatings * similar_movies)/(sum(np.abs(similar_movies))+0.01)
        similar_candidates = similar_candidates.append([predict_ratings])
    similar_candidates.index = corrMatrix.index
    similar_candidates = similar_candidates[~similar_candidates.index.isin(myRatings.index)]
    similar_candidates.columns = ['PredictScore']
    similar_candidates.sort_values(by='PredictScore',inplace=True,ascending=False)    
    return similar_candidates[:hm]


In [233]:
# recommend movies for Kris
rec_movie = recommend_movie(NAME='Kris',hm=10)
rec_movie

,PredictScore
Moon Man,0.990099


Recommended movie for Kris: Moon Man

In [229]:
# recommend movies for Mario
rec_movie = recommend_movie(NAME='Mario',hm=10)
rec_movie

,PredictScore
Thor,0.986944
Bullet Train,-0.984955
Dr. strange,-0.989528
Top Gun,-0.990099


Recommended movie for Mario: Thor

In [231]:
# recommend movies for Leo
rec_movie = recommend_movie(NAME='Leo',hm=10)
rec_movie

,PredictScore
Moon Man,0.000000
Minions,-1.389493
Top Gun,-2.487562


Recommended movie for Leo: Moon Man

In [232]:
# recommend movies for Mike
rec_movie = recommend_movie(NAME='Mike',hm=10)
rec_movie

,PredictScore
Moon Man,3.960396
Top Gun,-3.322259


Recommended movie for Mike: Moon Man